# Yksinkertainen kuvien luokittelija

Tämä muistikirja opastaa, kuinka voit luokitella kuvia esikoulutetun neuroverkon avulla.

**Mitä opit:**
- Kuinka ladata ja käyttää esikoulutettua mallia
- Kuvien esikäsittely
- Ennusteiden tekeminen kuvista
- Luottamusarvojen ymmärtäminen

**Käyttötapaus:** Tunnista esineitä kuvista (kuten "kissa", "koira", "auto" jne.)

---


## Vaihe 1: Tuo tarvittavat kirjastot

Tuodaan työkalut, joita tarvitsemme. Älä huoli, jos et vielä ymmärrä kaikkia näitä!


In [ ]:
# Core libraries
import numpy as np
from PIL import Image
import requests
from io import BytesIO

# TensorFlow for deep learning
try:
    import tensorflow as tf
    from tensorflow.keras.applications import MobileNetV2
    from tensorflow.keras.applications.mobilenet_v2 import preprocess_input, decode_predictions
    print("✅ TensorFlow loaded successfully!")
    print(f"   Version: {tf.__version__}")
except ImportError:
    print("❌ Please install TensorFlow: pip install tensorflow")

## Vaihe 2: Lataa esikoulutettu malli

Käytämme **MobileNetV2**-mallia, joka on jo koulutettu miljoonilla kuvilla.

Tätä kutsutaan **siirto-oppimiseksi** – käytetään mallia, jonka joku muu on kouluttanut!


In [ ]:
print("📦 Loading pre-trained MobileNetV2 model...")
print("   This may take a minute on first run (downloading weights)...")

# Load the model
# include_top=True means we use the classification layer
# weights='imagenet' means it was trained on ImageNet dataset
model = MobileNetV2(weights='imagenet', include_top=True)

print("✅ Model loaded!")
print(f"   The model can recognize 1000 different object categories")

## Vaihe 3: Apufunktiot

Luodaan funktiot kuvien lataamiseen ja valmisteluun malliamme varten.


In [ ]:
def load_image_from_url(url):
    """
    Load an image from a URL.
    
    Args:
        url: Web address of the image
        
    Returns:
        PIL Image object
    """
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))
    return img


def prepare_image(img):
    """
    Prepare an image for the model.
    
    Steps:
    1. Resize to 224x224 (model's expected size)
    2. Convert to array
    3. Add batch dimension
    4. Preprocess for MobileNetV2
    
    Args:
        img: PIL Image
        
    Returns:
        Preprocessed image array
    """
    # Resize to 224x224 pixels
    img = img.resize((224, 224))
    
    # Convert to numpy array
    img_array = np.array(img)
    
    # Add batch dimension (model expects multiple images)
    img_array = np.expand_dims(img_array, axis=0)
    
    # Preprocess for MobileNetV2
    img_array = preprocess_input(img_array)
    
    return img_array


def classify_image(img):
    """
    Classify an image and return top predictions.
    
    Args:
        img: PIL Image
        
    Returns:
        List of (class_name, confidence) tuples
    """
    # Prepare the image
    img_array = prepare_image(img)
    
    # Make prediction
    predictions = model.predict(img_array, verbose=0)
    
    # Decode predictions to human-readable labels
    # top=5 means we get the top 5 most likely classes
    decoded = decode_predictions(predictions, top=5)[0]
    
    # Convert to simpler format
    results = [(label, float(confidence)) for (_, label, confidence) in decoded]
    
    return results


print("✅ Helper functions ready!")

## Vaihe 4: Testaa esimerkkikuvilla

Kokeillaan luokitella joitakin kuvia internetistä!


In [ ]:
# Sample images to classify
# These are from Unsplash (free stock photos)
test_images = [
    {
        "url": "https://images.unsplash.com/photo-1514888286974-6c03e2ca1dba?w=400",
        "description": "A cat"
    },
    {
        "url": "https://images.unsplash.com/photo-1552053831-71594a27632d?w=400",
        "description": "A dog"
    },
    {
        "url": "https://images.unsplash.com/photo-1511919884226-fd3cad34687c?w=400",
        "description": "A car"
    },
]

print(f"🧪 Testing on {len(test_images)} images...")
print("=" * 70)

### Luokittele Jokainen Kuva


In [ ]:
for i, img_data in enumerate(test_images, 1):
    print(f"\n📸 Image {i}: {img_data['description']}")
    print("-" * 70)
    
    try:
        # Load image
        img = load_image_from_url(img_data['url'])
        
        # Display image
        display(img.resize((200, 200)))  # Show smaller version
        
        # Classify
        results = classify_image(img)
        
        # Show predictions
        print("\n🎯 Top 5 Predictions:")
        for rank, (label, confidence) in enumerate(results, 1):
            # Create a visual bar
            bar_length = int(confidence * 50)
            bar = "█" * bar_length
            
            print(f"  {rank}. {label:20s} {confidence*100:5.2f}% {bar}")
        
    except Exception as e:
        print(f"❌ Error: {e}")

print("\n" + "=" * 70)

## Vaihe 5: Kokeile omia kuviasi!

Korvaa alla oleva URL millä tahansa kuvan URL-osoitteella, jonka haluat luokitella.


In [ ]:
# Try your own image!
# Replace this URL with any image URL
custom_image_url = "https://images.unsplash.com/photo-1472491235688-bdc81a63246e?w=400"  # A flower

print("🖼️  Classifying your custom image...")
print("=" * 70)

try:
    # Load and show image
    img = load_image_from_url(custom_image_url)
    display(img.resize((300, 300)))
    
    # Classify
    results = classify_image(img)
    
    # Show results
    print("\n🎯 Top 5 Predictions:")
    print("-" * 70)
    for rank, (label, confidence) in enumerate(results, 1):
        bar_length = int(confidence * 50)
        bar = "█" * bar_length
        print(f"  {rank}. {label:20s} {confidence*100:5.2f}% {bar}")
    
    # Highlight top prediction
    top_label, top_confidence = results[0]
    print("\n" + "=" * 70)
    print(f"\n🏆 Best guess: {top_label} ({top_confidence*100:.2f}% confident)")
    
except Exception as e:
    print(f"❌ Error: {e}")
    print("   Make sure the URL points to a valid image!")

## 💡 Mitä juuri tapahtui?

1. **Latasimme esikoulutetun mallin** - MobileNetV2 on koulutettu miljoonilla kuvilla
2. **Esikäsittelimme kuvat** - Muokkasimme ja formatoimme ne mallia varten
3. **Malli teki ennusteita** - Se tuotti todennäköisyyksiä 1000 objektiluokalle
4. **Dekoodasimme tulokset** - Muutimme numerot ihmisten ymmärtämiin etiketteihin

### Ymmärrä luottamusarvot

- **90-100%**: Erittäin varma (melkein varmasti oikein)
- **70-90%**: Varma (todennäköisesti oikein)
- **50-70%**: Jossain määrin varma (saattaa olla oikein)
- **Alle 50%**: Ei kovin varma (epävarma)

### Miksi ennusteet voivat olla vääriä?

- **Epätavallinen kulma tai valaistus** - Malli on koulutettu tyypillisillä valokuvilla
- **Useita objekteja** - Malli odottaa yhtä pääobjektia
- **Harvinaiset objektit** - Malli tuntee vain 1000 kategoriaa
- **Huonolaatuinen kuva** - Epätarkat tai pikselöityneet kuvat ovat haastavampia

---


## 🚀 Seuraavat askeleet

1. **Kokeile eri kuvia:**
   - Etsi kuvia [Unsplash](https://unsplash.com)-sivustolta
   - Napsauta hiiren oikealla → "Kopioi kuvan osoite" saadaksesi URL-osoitteen

2. **Kokeile:**
   - Mitä tapahtuu abstraktin taiteen kanssa?
   - Voiko se tunnistaa esineitä eri kulmista?
   - Miten se käsittelee useita esineitä?

3. **Opi lisää:**
   - Tutustu [Tietokonenäkö-opetuksiin](../lessons/4-ComputerVision/README.md)
   - Opi kouluttamaan oma kuvantunnistusmalli
   - Ymmärrä, miten CNN:t (Convolutional Neural Networks) toimivat

---

## 🎉 Onnittelut!

Rakensit juuri kuvantunnistimen käyttäen huippuluokan neuroverkkoa!

Tämä sama tekniikka mahdollistaa:
- Google Kuvat (valokuvien järjestäminen)
- Itseohjautuvat autot (esineiden tunnistaminen)
- Lääketieteellinen diagnostiikka (röntgenkuvien analysointi)
- Laadunvalvonta (vikojen havaitseminen)

Jatka tutkimista ja oppimista! 🚀



---

**Vastuuvapauslauseke**:  
Tämä asiakirja on käännetty käyttämällä tekoälypohjaista käännöspalvelua [Co-op Translator](https://github.com/Azure/co-op-translator). Vaikka pyrimme tarkkuuteen, huomioithan, että automaattiset käännökset voivat sisältää virheitä tai epätarkkuuksia. Alkuperäistä asiakirjaa sen alkuperäisellä kielellä tulisi pitää ensisijaisena lähteenä. Kriittisen tiedon osalta suositellaan ammattimaista ihmiskäännöstä. Emme ole vastuussa väärinkäsityksistä tai virhetulkinnoista, jotka johtuvat tämän käännöksen käytöstä.
